# Agentchat Module

AutoGen offers several classes allowing developers to work with LLMs and solve tasks, particularly in the domain of mathematics and retrieval-based tasks.

In [1]:
import os
import json
import openai
import autogen
import tempfile
import chromadb
from pathlib import Path

import sys
sys.path.append('./')

from utils.api_utils import config_list_from_dotenv

config_list = config_list_from_dotenv()

models to use:  ['gpt-4', 'gpt-3.5-turbo']


retrieve_assistant_agent
The RetrieveAssistantAgent belongs to the agentchat.contrib.retrieve_assistant_agent module and is designed to solve tasks with LLMs, including suggesting Python code blocks and debugging. This agent does not execute code by default and expects the user to execute the code. This class is a subclass of AssistantAgent, configured with a default system message to solve tasks with LLMs.

Purpose: The RetrieveAssistantAgent is a specialized agent designed to solve tasks with Language Learning Models (LLMs) like GPT. It is configured with a default system message and is specialized in suggesting Python code blocks and assisting with debugging. By default, human_input_mode is set to "NEVER", and code_execution_config is set to False, meaning this agent does not execute code by default and expects the user to execute the code. It is particularly useful when the user needs suggestions or guidance in writing or debugging Python code, but the execution of the code is left to the user.

retrieve_user_proxy_agent
The RetrieveUserProxyAgent class is part of the agentchat.contrib.retrieve_user_proxy_agent module and seems to be designed to interact with users, asking for human inputs every time a message is received based on its configuration. It also allows the generation of initial messages with given problems and prompts and can be configured with various parameters like the task type, client, docs_path, collection_name, and model to use for the retrieve chat.

### retrieve_assistant_agent

The `RetrieveAssistantAgent` belongs to the `agentchat.contrib.retrieve_assistant_agent` module and is designed to solve tasks with LLMs, including suggesting Python code blocks and debugging. This agent does not execute code by default and expects the user to execute the code. This class is a subclass of AssistantAgent, configured with a default system message to solve tasks with LLMs.

**Purpose:**
The RetrieveAssistantAgent is a specialized agent designed to solve tasks with Language Learning Models (LLMs) like GPT. It is configured with a default system message and is specialized in suggesting Python code blocks and assisting with debugging. By default, human_input_mode is set to "NEVER", and `code_execution_config` is set to False, meaning this agent does not execute code by default and expects the user to execute the code. It is particularly useful when the user needs suggestions or guidance in writing or debugging Python code, but the execution of the code is left to the user.


#### retrieve_user_proxy_agent
The `RetrieveUserProxyAgent` class is part of the `agentchat.contrib.retrieve_user_proxy_agent` module and seems to be designed to interact with users, asking for human inputs every time a message is received based on its configuration. It also allows the generation of initial messages with given problems and prompts and can be configured with various parameters like the task type, client, docs_path, collection_name, and model to use for the retrieve chat.

In [4]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Start logging
autogen.ChatCompletion.start_logging()

# Create an instance of RetrieveAssistantAgent
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
)

path = Path(os.getcwd(), 'docs')
str(path)

client = chromadb.PersistentClient(path=f"{os.getcwd()}/chromadb")

# Create an instance of RetrieveUserProxyAgent
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "default",
        "docs_path": str(path), 
        "chunk_token_size": 2000,
        "model": config_list[0]["model"],
        "client": client, 
        "embedding_model": "all-mpnet-base-v2",
    },
)

In [5]:
# NOTE: Delete dir each instantiation
#  client.delete_collection('autogen-docs')

In [6]:
ragproxyagent.retrieve_docs("some query string")

Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	BLUE is the enterprise’s integrated, flexible and interoperable platform that will anchor the scale- ...
Failed to split docs with must_break_at_empty_line being True, set to False.
Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


doc_ids:  [['doc_0', 'doc_1', 'doc_2']]


In [7]:
# From a txt document 
user_question = "Who was the blacksmith?"
ragproxyagent.initiate_chat(assistant, problem=user_question) 

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


doc_ids:  [['doc_2', 'doc_0', 'doc_1']]
Adding doc_id doc_2 to context.
Adding doc_id doc_0 to context.
Adding doc_id doc_1 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
If user's intent is code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

If user's intent is question answering, you must give as short an answer as possible.

### math_user_proxy_agent

The `MathUserProxyAgent` is a part of the `agentchat.contrib.math_user_proxy_agent` module, and it is designed to handle math problems. It seems to be experimental at this stage. This class can generate initial messages, execute Python code, and run queries through Wolfram Alpha to solve mathematical problems.